In [1]:
import pandas as pd
from openai import OpenAI
from scripts.config import *
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)



cuda


In [2]:
def load_dataset(path_to_dataset):
    return pd.read_csv(path_to_dataset)

def extract_similar_descriptions(e, embeddings, k):
    similarities = cosine_similarity([e], embeddings)[0]
    top_indices = np.argsort(similarities)[-k:][::-1]
    return top_indices
    
def compute_embeddings_distance(embedding_str, k, df):
    #df = load_dataset(path_to_dataset)

    # QUitamos del dataset la misma fila con la que estamos trabajando
    df = df[df["embeddings_ingles"]!= embedding_str]

    def parse_embedding(embedding_str):
        embedding_str = embedding_str.strip("[]")
        return np.array(embedding_str.split(), dtype=np.float32)
    
    embedding = parse_embedding(embedding_str)
    df["embeddings_ingles"] = df["embeddings_ingles"].apply(lambda x: parse_embedding(x))

    df_pos = df[df["label"]==2]
    df_neu = df[df["label"]==1]
    df_neg = df[df["label"]==0]

    pos_description_embeddings = np.vstack(df_pos["embeddings_ingles"].values)
    pos_nearest_descriptions = extract_similar_descriptions(embedding, pos_description_embeddings, k)
    pos_result = df_pos.iloc[pos_nearest_descriptions]["translation"]

    neu_description_embeddings = np.vstack(df_neu["embeddings_ingles"].values)
    neu_nearest_descriptions = extract_similar_descriptions(embedding, neu_description_embeddings, k)
    neu_result = df_neu.iloc[neu_nearest_descriptions]["translation"]

    neg_description_embeddings = np.vstack(df_neg["embeddings_ingles"].values)
    neg_nearest_descriptions = extract_similar_descriptions(embedding, neg_description_embeddings, k)
    neg_result = df_neg.iloc[neg_nearest_descriptions]["translation"]
    
    pos_result, neu_result, neg_result = pos_result.to_list(), neu_result.to_list(), neg_result.to_list()

    pos_str = ""
    for i in range(len(pos_result)):
        pos_str += str(i+1) + ". TEXT:" + pos_result[i] + " LABEL: POS\n"

    neu_str = ""
    for i in range(len(neu_result)):
        neu_str += str(i+1) + ". TEXT:" + neu_result[i] + " LABEL: NEU\n"

    neg_str = ""
    for i in range(len(neg_result)):
        neg_str += str(i+1) + ". TEXT:" + neg_result[i] + " LABEL: NEG\n"

    return pos_str, neu_str, neg_str

mapping = {"NEG":0, "NEU":1, "POS":2}
reverse_mapping = {0:"NEG", 1: "NEU", 2:"POS"}

def label2int(label):
    return mapping[label]

In [9]:
data = load_dataset("../data/data.csv")

In [10]:
# train_df = load_dataset("../data/train_paraphrased.csv")
# test_df = load_dataset("../data/test_paraphrased.csv")
# val_df = load_dataset("../data/val_paraphrased.csv")
# dev_df = load_dataset("../data/dev_paraphrased.csv")
# train_df = pd.concat([train_df, dev_df]).reset_index()

train_df, test_df = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data['label'])
#train_df, val_df = train_test_split(train_df, test_size=1/8, random_state=RANDOM_SEED, stratify=train_df['label'])


In [12]:
def initialize_client(key):

  client = OpenAI(
    api_key = key
  )

  return client

def predict_val_dataset(post, keyword, pos, neg, neu, client):
  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    store=True,
    messages=[
      {"role": "developer", "content": DEVELOPER_PROMPT},
      {"role" : "user", "content": f"{POST_PROMPT} Keyword: '{keyword}' Post to classify: '{post}'. {POST_PROMPT1} {CONTEXT_PROMPT_POS} {pos}. {CONTEXT_PROMPT_NEG} {neg}. {CONTEXT_PROMPT_NEU} {neu}. {CONTEXT_PROMPT}"}
    ]
  )

  return completion.choices[0].message.content

client = initialize_client(OPENAI_API_KEY)

## Inglés

In [6]:
EMB_COL = "embeddings_ingles"
TEXT_COL = "translation"

In [7]:
DEVELOPER_PROMPT = "You are an assistant tasked with classifying Reddit posts based on the sentiment they express towards a given Keyword. There are three possible classes: Positive (POS), Negative (NEG), or Neutral (NEU)."
CONTEXT_PROMPT_POS = "To give you more context, I will provide three lists containing the most similar texts to the post to be classified. Examples of positive posts: "
CONTEXT_PROMPT_NEG = "Examples of negative posts: "
CONTEXT_PROMPT_NEU = "Examples of neutral posts: "
CONTEXT_PROMPT = "Remember, return only the label: POS, NEG, or NEU."
POST_PROMPT = "Classify this Reddit post as POS, NEG, or NEU based on its sentiment or polarity to this given keyword. "
POST_PROMPT1 = "Return only the label. "

#### VAL RESULTS

In [29]:
preds = []
for i, emb in enumerate(val_df["embeddings_ingles"]):
    print(i)
    post = val_df["translation"][i]
    keyword = val_df["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 10, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds.append(response)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
true_labels_val = val_df["label"].apply(lambda x: reverse_mapping[x])

In [31]:
y_pred_val = np.array(preds)
y_true_val = np.array(true_labels_val)
print(y_pred_val.shape, y_true_val.shape)
print(classification_report(y_true_val, y_pred_val))

(721,) (721,)
              precision    recall  f1-score   support

         NEG       0.50      0.72      0.59       244
         NEU       0.71      0.48      0.57       406
         POS       0.35      0.48      0.40        71

    accuracy                           0.56       721
   macro avg       0.52      0.56      0.52       721
weighted avg       0.60      0.56      0.56       721



#### TEST RESULTS

In [32]:
preds = []
for i, emb in enumerate(test_df[EMB_COL]):
    print(i)
    post = test_df[TEXT_COL][i]
    keyword = test_df["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 10, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds.append(response)
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
true_labels_test = test_df["label"].apply(lambda x: reverse_mapping[x])

In [34]:
y_pred_test = np.array(preds)
y_true_test = np.array(true_labels_test)
print(y_pred_test.shape, y_true_test.shape)
print(classification_report(y_true_test, y_pred_test))

(1442,) (1442,)
              precision    recall  f1-score   support

         NEG       0.52      0.76      0.62       487
         NEU       0.76      0.52      0.62       812
         POS       0.39      0.48      0.43       143

    accuracy                           0.60      1442
   macro avg       0.56      0.59      0.56      1442
weighted avg       0.64      0.60      0.60      1442



### TASK 1 RESULTS

In [3]:
task_1 = load_dataset("../data/translated_dataset_task1.csv")

In [15]:
task_1.head()

,Unnamed: 0,id,country,keyword,post content,translation,embeddings_ingles
0,0,A1642,Argentina,asexual,"demisexualidad del sufijo en inglés demi-, «a ...","Demisexuality, from the English suffix demi-, ...",[ 6.89718127e-02 2.06939112e-02 9.28649493e-...
1,1,A1114,Argentina,asexual,"a ver bro, te cuento algo peor? fui a tomar al...","Let's see, bro, let me tell you something wors...",[ 5.15531227e-02 5.59564941e-02 -3.76055054e-...
2,2,A4434,Argentina,asexual,"pensé y no, acá se da a entender que está mira...","I thought and no, it is understood here that i...",[ 7.19494699e-03 -7.63614848e-02 -6.96012303e-...
3,3,A1431,Argentina,asexual,¿que tal les fue en las reuniones de ex alumno...,How did it go for you at the reunions of the e...,[-6.91065341e-02 1.23082567e-02 3.59575190e-...
4,4,A4436,Argentina,asexual,creo que el creador afirmó que era `asexual` e...,I believe the creator stated that they were 'a...,[-5.62392920e-02 4.87700477e-02 -7.24150315e-...


In [ ]:
preds_task1 = []
for i, emb in enumerate(task_1[EMB_COL]):
    print(i)
    post = task_1[TEXT_COL][i]
    keyword = task_1["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 2, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds_task1.append(response)
    

0


### TASK 2 RESULTS

In [17]:
task_2 = load_dataset("../data/translated_dataset_task2.csv")

In [20]:
task_2.head()

,Unnamed: 0,id,country,keyword,post content,translation,embeddings_ingles
0,0,R0000,Uruguay,gay,"creo q es esta, la de u r gay y otra más que n...","I think it's this one, the ""u r gay"" and anoth...",[-8.81991256e-03 -8.53167288e-03 -8.68314654e-...
1,1,R0001,Uruguay,transgénero,que tal peligroso es ser gay/trans? les tengo ...,How dangerous is it to be gay/trans? I have a ...,[ 4.35901731e-02 -2.71045119e-02 -4.79880422e-...
2,2,R0002,Uruguay,transexual,es un mundo de diferencia entre ser homosexual...,There is a world of difference between being h...,[ 2.06695944e-02 5.58363367e-03 -4.17829044e-...
3,3,R0003,Uruguay,lesbiana,te cuento mi perspectiva como lesbiana viviend...,I'll share my perspective as a lesbian living ...,[ 4.77791168e-02 -6.87944666e-02 3.10976245e-...
4,4,R0004,Uruguay,transexuales,"no es peligroso, en general. pueden sufrir dis...","It's not dangerous, in general. They may suffe...",[ 2.88035162e-02 9.82037280e-03 -8.06938037e-...


In [ ]:
preds_task2 = []
for i, emb in enumerate(task_2[EMB_COL]):
    print(i)
    post = task_2[TEXT_COL][i]
    keyword = task_2["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 2, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds_task2.append(response)
    

## ESPAÑOL

In [ ]:
EMB_COL = "embeddings"
TEXT_COL = "post content"

In [ ]:
DEVELOPER_PROMPT = "Eres un asistente encargado de clasificar posts de Reddit de diferentes zonas de SudAmérica según el sentimiento que expresan hacia una palabra clave dada. Hay tres posibles clases: Positivo (POS), Negativo (NEG) o Neutral (NEU)."
CONTEXT_PROMPT_POS = "Para darte más contexto, te proporcionaré tres listas que contienen los textos más similares a la publicación que se va a clasificar. Ejemplos de publicaciones positivas: "
CONTEXT_PROMPT_NEG = "Ejemplos de posts negativas: "
CONTEXT_PROMPT_NEU = "Ejemplos de posts neutrales: "
CONTEXT_PROMPT = "Recuerda, devuelve solo la etiqueta: POS, NEG o NEU."
POST_PROMPT = "Clasifica esta publicación de Reddit como POS, NEG o NEU en función de su sentimiento o polaridad hacia la palabra clave dada. "
POST_PROMPT1 = "Devuelve solo la etiqueta. "

In [ ]:
def load_dataset(path_to_dataset):
    return pd.read_csv(path_to_dataset, index_col = 0)

def extract_similar_descriptions(e, embeddings, k):
    similarities = cosine_similarity([e], embeddings)[0]
    top_indices = np.argsort(similarities)[-k:][::-1]
    return top_indices
    
def compute_embeddings_distance(embedding_str, k, df):
    #df = load_dataset(path_to_dataset)

    # QUitamos del dataset la misma fila con la que estamos trabajando
    df = df[df["embeddings"]!= embedding_str]

    def parse_embedding(embedding_str):
        embedding_str = embedding_str.strip("[]")
        return np.array(embedding_str.split(), dtype=np.float32)
    
    embedding = parse_embedding(embedding_str)
    df["embeddings"] = df["embeddings"].apply(lambda x: parse_embedding(x))

    df_pos = df[df["label"]==2]
    df_neu = df[df["label"]==1]
    df_neg = df[df["label"]==0]

    pos_description_embeddings = np.vstack(df_pos["embeddings"].values)
    pos_nearest_descriptions = extract_similar_descriptions(embedding, pos_description_embeddings, k)
    pos_result = df_pos.iloc[pos_nearest_descriptions]["post content"]

    neu_description_embeddings = np.vstack(df_neu["embeddings"].values)
    neu_nearest_descriptions = extract_similar_descriptions(embedding, neu_description_embeddings, k)
    neu_result = df_neu.iloc[neu_nearest_descriptions]["post content"]

    neg_description_embeddings = np.vstack(df_neg["embeddings"].values)
    neg_nearest_descriptions = extract_similar_descriptions(embedding, neg_description_embeddings, k)
    neg_result = df_neg.iloc[neg_nearest_descriptions]["post content"]
    

    return pos_result.to_list(), neu_result.to_list(), neg_result.to_list()

mapping = {"NEG":0, "NEU":1, "POS":2}
reverse_mapping = {0:"NEG", 1: "NEU", 2:"POS"}

def label2int(label):
    return mapping[label]

### VAL RESULTS

In [ ]:
preds = []
for i, emb in enumerate(val_df[EMB_COL]):
    print(i)
    post = val_df[TEXT_COL][i]
    keyword = val_df["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 2, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds.append(response)

true_labels_val = val_df["label"].apply(lambda x: reverse_mapping[x])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
y_pred_val = np.array(preds)
y_true_val = np.array(true_labels_val)
print(y_pred_val.shape, y_true_val.shape)
print(classification_report(y_true_val, y_pred_val))

(721,) (721,)
              precision    recall  f1-score   support

         NEG       0.51      0.82      0.63       244
         NEU       0.77      0.45      0.56       406
         POS       0.33      0.45      0.38        71

    accuracy                           0.57       721
   macro avg       0.54      0.57      0.53       721
weighted avg       0.64      0.57      0.57       721



### TEST RESULTS

In [ ]:
preds = []
for i, emb in enumerate(test_df[EMB_COL]):
    print(i)
    post = test_df[TEXT_COL][i]
    keyword = test_df["keyword"][i]
    pos, neu, neg = compute_embeddings_distance(emb, 2, train_df)
    response = predict_val_dataset(post, keyword, pos, neg, neu, client)
    preds.append(response)

true_labels_test = test_df["label"].apply(lambda x: reverse_mapping[x])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
y_pred_test = np.array(preds)
y_true_test = np.array(true_labels_test)
print(y_pred_test.shape, y_true_test.shape)
print(classification_report(y_true_test, y_pred_test))

(1442,) (1442,)
              precision    recall  f1-score   support

         NEG       0.51      0.80      0.62       487
         NEU       0.74      0.47      0.57       812
         POS       0.33      0.38      0.35       143

    accuracy                           0.57      1442
   macro avg       0.53      0.55      0.52      1442
weighted avg       0.62      0.57      0.57      1442

